In [2]:
#from Sensor_Modelling\Functions\Variables import se_Magnitude_Limit

from Sensor_Modelling.Functions.Variables import se_Magnitude_Limit

In [3]:
se_Magnitude_Limit

6

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import os

# Initialise variables
from Sensor_Modelling.Functions.Variables import *

# Import Functions
from Sensor_Modelling.Functions.SensorModelling_Functions import *
from Sensor_Modelling.Functions.SensorModelling_Functions_2 import *
from Sensor_Modelling.Functions.Other_Functions import *

In [3]:
def main(debug = False):
    se_count = se_get_var_value()
    
    # Import Catalogue
    CATALOGUE = pd.read_csv("Sensor_Modelling/Preprocessing/Simulation_HYG.csv")
    CATALOGUE.sort_values('Mag', inplace = True)
    CATALOGUE.set_index('StarID', drop = False, inplace = True)
    # The Star Catalogue being used is the HYG Database
    # StarID: The database primary key from a larger "master database" of stars
    # Mag: The star's apparent visual magnitude
    # RA, Dec: The star's right ascension and declination, for epoch 2000.0 (Unit: RA - hrs; Dec - degrees)
    print(CATALOGUE.head())
    if debug: print("Catalogue read")
    
    # The following pandas dataframe 'IMG_Data' contains the coordinates of the stars in the sensor frame
    IMG_Data = generateImageDataframe(CATALOGUE, RA, Dec, Circular_FOV, se_Magnitude_Limit, ra_hrs = False)
    IMG_Data['Roll'] = Roll
    print(IMG_Data.head())
    IMG_Data = IMG_Data.apply(generateCoordinates3D, result_type = 'expand', axis=1).rename(columns = {0 : "x", 1 : "y", 2 : "z", 3 : "Mag"})
    print(IMG_Data.head())
    IMG_Data = IMG_Data.apply(imageFrameCoordinates, result_type = 'expand', axis=1).rename(columns = {0 : "x", 1 : "y", 2 : "Mag"})
    print(IMG_Data.head())
    if debug: print('Dataframe generated')
    
    # Generating the image
    Image = np.zeros((CMOS_Width, CMOS_Length))
    #generateImage(Image, IMG_Data)

    # Trimming the data to within the boundary
    IMG_Data = IMG_Data[IMG_Data['x'] < pixel_size * CMOS_Length / 2]
    IMG_Data = IMG_Data[IMG_Data['x'] > - pixel_size * CMOS_Length / 2]
    IMG_Data = IMG_Data[IMG_Data['y'] < pixel_size * CMOS_Width / 2]
    IMG_Data = IMG_Data[IMG_Data['y'] > - pixel_size * CMOS_Width / 2]
    
    # Converting values to arrays - x, y coordinates and brightness magnitude
    y = IMG_Data.iloc[:,0].values / pixel_size
    x = IMG_Data.iloc[:,1].values / pixel_size
    b = IMG_Data.iloc[:,2].values
    
    # n is the number of stars in the image
    n = IMG_Data.shape[0]
    
    
    for i in np.arange(n):
        for j in np.arange(int(x[i] - check_radius) if int(x[i] - check_radius) > 0 else 0, int(x[i] + check_radius) if int(x[i] + check_radius) < CMOS_Width else CMOS_Width):
            for k in np.arange(int(y[i] - check_radius) if int(y[i] - check_radius) > 0 else 0, int(y[i] + check_radius) if int(y[i] + check_radius) < CMOS_Length else CMOS_Length):
                Image[j][k] = Image[j][k] + f(j, k, H(b[i]), sigma, (x[i] + CMOS_Width / 2), (y[i] + CMOS_Length / 2))
    
    if debug: print('Image generated')
        
    mpimg.imsave("Sensor_Modelling/Output/Image.png", Image, cmap='gray', vmax = 255)
    se_dirName = 'Sensor_Modelling/Output/History/' + "{0:0=3d}".format(se_count)
    os.mkdir(se_dirName)
    mpimg.imsave(os.path.join(se_dirName , 'Image.png'), Image, cmap='gray', vmax = 255)


no_loops = 1
for i in np.arange(no_loops):
    RA_hr = se_Dataset[i][0]
    RA = RA_hr * 360 / 24 #In Degrees
    Dec = se_Dataset[i][1]
    Roll = se_Dataset[i][2]
    main(debug = True)

        StarID         RA        Dec   Mag
StarID                                    
23440    23440   6.752569 -16.713143 -1.44
21936    21936   6.399192 -52.695718 -0.62
50628    50628  14.261208  19.187270 -0.05
51926    51926  14.661361 -60.835147 -0.01
65795    65795  18.615607  38.782993  0.03
Catalogue read
      Ref_RA   Ref_Dec  Star_ID        RA        Dec   Mag  Roll
0   5.603559 -1.201917  23440.0  6.752569 -16.713143 -1.44   0.0
6   5.603559 -1.201917  17301.0  5.242298  -8.201639  0.18   0.0
9   5.603559 -1.201917  19984.0  5.919525   7.407036  0.45   0.0
13  5.603559 -1.201917  15108.0  4.598667  16.509762  0.87   0.0
27  5.603559 -1.201917  17947.0  5.418852   6.349735  1.64   0.0
           x         y         z   Mag
0   0.283778 -0.306705  0.908517 -1.44
6  -0.093471 -0.163294  0.982140  0.18
9   0.081936  0.108159  0.990751  0.45
13 -0.249334  0.264697  0.931541  0.87
27 -0.048041  0.089750  0.994805  1.64
            x          y   Mag
0  -12.494109  13.503534 -1.4

In [5]:
4 if 2 > 3 else 5

5